In [ ]:
import json

In [ ]:
import boto3

In [ ]:
aws_region = ''
aws_account_id = ''

In [ ]:
sqs_queue_url = "https://sqs.{}.amazonaws.com/{}/nept-mlops-dev-gremlin-csv-loader".format(aws_region, aws_account_id)
s3_bucket = "nept-mlops-dev-{}".format(aws_account_id)

In [ ]:
s3_client = boto3.client('s3')
sqs_client = boto3.client('sqs')

In [ ]:
vertex_file_content = """~id, name:String, age:Int, lang:String, interests:String[], ~label
v1, "marko", 29, , "sailing;graphs", person
v2, "lop", , "java", , software
"""
edge_file_content = """~id, ~from, ~to, ~label, weight:Double
e1, v1, v2, created, 0.4
"""

In [ ]:
gremlin_csv_prefix = 'gremlin-csv'

In [ ]:
vertex_file_s3_key = "{}/vertexes.csv".format(gremlin_csv_prefix)
edge_file_s3_key = "{}/edges.csv".format(gremlin_csv_prefix)

In [ ]:
s3_client.put_object(Body=vertex_file_content.encode('utf-8'), Bucket=s3_bucket, Key=vertex_file_s3_key)

In [ ]:
s3_client.put_object(Body=edge_file_content.encode('utf-8'), Bucket=s3_bucket, Key=edge_file_s3_key)

In [ ]:
sqs_message_to_send = {
    "vertex_files": ["s3://{}/{}".format(s3_bucket, vertex_file_s3_key)],
    "edge_files": ["s3://{}/{}".format(s3_bucket, edge_file_s3_key)]
}

In [ ]:
sqs_client.send_message(
    QueueUrl=sqs_queue_url,
    MessageBody=json.dumps(sqs_message_to_send)
)

In [ ]:
received_sqs_message = sqs_client.receive_message(
    QueueUrl=sqs_queue_url,
    MaxNumberOfMessages=1,
    WaitTimeSeconds=20)['Messages'][0]

In [ ]:
receipt_handle = received_sqs_message['ReceiptHandle']

In [ ]:
received_message_json = json.loads(received_sqs_message['Body'])

In [ ]:
received_message_json

In [ ]:
sqs_client.delete_message(QueueUrl=sqs_queue_url, ReceiptHandle=receipt_handle)